In [1]:
import dash
from dash import html, dcc, Input, Output, State, dash_table
import pandas as pd
import plotly.express as px
import pymongo
from bson.objectid import ObjectId
from epicsdb import connectMongo, pushData_list
import json

username = 'bl531'
pwd = 'bl531'

In [2]:
# get the latest database from cloud
def getDB(username, pwd, dbname, collectionName = 'epicsPVs'):
    client = connectMongo('bl531','bl531')
    collection = client.get_database(dbname).get_collection(collectionName)
    df = pd.DataFrame(list(collection.find()))
    return df

In [ ]:
# load database from cloud server
client = connectMongo('bl531','bl531')
collection = client.epicsHappi.epicsPVs

In [6]:
# create the layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets,
                suppress_callback_exceptions=True)

app.layout = html.Div([
    html.H1('EPICS Database', style={'textAlign': 'Center'}),
    # interval activated once/week or when page refreshed
    dcc.Interval(id='interval_db', interval=86400000 * 7, n_intervals=0),
    html.Div(id='mongo-datatable', children=[]),

    html.Div([
        html.Div(id='pie-graph', className='five columns'),
        html.Div(id='hist-graph', className='six columns'),
    ], className='row'),
    dcc.Store(id='changed-cell')
])

# Display Datatable with data from Mongo database
@app.callback(Output('mongo-datatable', component_property='children'),
              Input('interval_db', component_property='n_intervals')
              )
def populate_datatable(n_intervals):
    # Convert the Collection (table) date to a pandas DataFrame
    df = pd.DataFrame(list(collection.find()))
    # Convert id from ObjectId to string so it can be read by DataTable
    df['_id'] = df['_id'].astype(str)

    return [
        dash_table.DataTable(
            id='our-table',
            data=df.to_dict('records'),
            # columns=[{'id':p, 'name':p, 'editable':True} for p in df if p!='_id']
            columns=[{'id': p, 'name': p, 'editable': False} if p == '_id'
                     else {'id': p, 'name': p, 'editable': True}
                     for p in df],
        ),
    ]

# store the row id and column id of the cell that was updated
app.clientside_callback(
    """
    function (input,oldinput) {
        if (oldinput != null) {
            if(JSON.stringify(input) != JSON.stringify(oldinput)) {
                for (i in Object.keys(input)) {
                    newArray = Object.values(input[i])
                    oldArray = Object.values(oldinput[i])
                    if (JSON.stringify(newArray) != JSON.stringify(oldArray)) {
                        entNew = Object.entries(input[i])
                        entOld = Object.entries(oldinput[i])
                        for (const j in entNew) {
                            if (entNew[j][1] != entOld[j][1]) {
                                changeRef = [i, entNew[j][0]] 
                                break        
                            }
                        }
                    }
                }
            }
            return changeRef
        }
    }    
    """,
    Output('changed-cell', 'data'),
    Input('our-table', 'data'),
    State('our-table', 'data_previous')
)

# Update MongoDB and create the graphs
@app.callback(
    Output("pie-graph", "children"),
    Output("hist-graph", "children"),
    Input("changed-cell", "data"),
    Input("our-table", "data"),
)
def update_d(cc, tabledata):
    if cc is None:
        # Build the Plots
        pie_fig = px.pie(tabledata, values='quantity', names='day')
        hist_fig = px.histogram(tabledata, x='department', y='quantity')
    else:
        # print(f'changed cell: {cc}')
        # print(f'Current DataTable: {tabledata}')
        x = int(cc[0])

        # update the external MongoDB
        row_id = tabledata[x]['_id']
        col_id = cc[1]
        new_cell_data = tabledata[x][col_id]
        collection.update_one({'_id': ObjectId(row_id)},
                                {"$set": {col_id: new_cell_data}})
        # Operations guide - https://docs.mongodb.com/manual/crud/#update-operations

        # pie_fig = px.pie(tabledata, values='quantity', names='day')
        # hist_fig = px.histogram(tabledata, x='department', y='quantity')

    return dcc.Graph(figure=pie_fig), dcc.Graph(figure=hist_fig)

In [11]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Mar/2023 09:42:06] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/dash/dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "/home/bl531/miniconda3/lib/python3.10/site-packages/dash/_callback.py", line 151, in add_context
    output_value = func(*func_arg

127.0.0.1 - - [16/Mar/2023 09:42:07] "POST /_dash-update-component HTTP/1.1" 500 -


In [3]:
df = getDB(username, pwd, 'epicsHappi', 'epicsPVs')
df2 = df.iloc[0:29]

In [6]:
df2

,_id,active,args,beamline,creation,detailed_screen,device_class,documentation,embedded_screen,engineering_screen,...,macros,name,parent,prefix,stand,system,type,z,unit,port
0,6412404f23d463e59014d013,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,tested AW,None,None,...,None,bl531_m101_pitch_mm,None,bl531_esp300:m101_pitch_mm,None,None,OphydItem,-1.0,mm,"ttyUSB300, axis1"
1,6412405023d463e59014d014,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,tested AW,None,None,...,None,bl531_m101_bend_um,None,bl531_esp300:m101_bend_um,None,None,OphydItem,-1.0,um,"ttyUSB300, axis2"
2,6412405023d463e59014d015,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,tested aW,None,None,...,None,bl531_m101_slit_st,None,bl531_esp300:m101_slit_mm,None,None,OphydItem,-1.0,mm,"ttyUSB300, axis3"
3,6412405023d463e59014d016,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,tested AW,None,None,...,None,bl531_saxs_mono_angle_deg,None,bl531_xps1:mono_angle_deg,None,None,OphydItem,-1.0,deg,"192.168.10.25, axis1"
4,6412405023d463e59014d017,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_mono_height_mm,None,bl531_xps1:mono_height_mm,None,None,OphydItem,-1.0,mm,"192.168.10.25, axis2"
5,6412405023d463e59014d018,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_A,None,DMC01:A,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit A"
6,6412405023d463e59014d019,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_B,None,DMC01:B,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit B"
7,6412405023d463e59014d01a,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_C,None,DMC01:C,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit C"
8,6412405023d463e59014d01b,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531:saxs:hs_slit:D,None,DMC01:D,None,None,motor,-1.0,mm,"192.168.10.21, slit D"
9,6412405023d463e59014d01c,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_E,None,DMC01:E,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit E"


In [ ]:
# get the latest database from cloud
df = getDB(username, pwd, 'epicHappi_mod', 'epicPVs')

In [7]:
# push the database into cloud
pushData_list(username, pwd, 'epicsHappi_mod', df2.to_dict('records'))

No such fields
Uploading bl531_m101_pitch_mm
No such fields
Uploading bl531_m101_bend_um
No such fields
Uploading bl531_m101_slit_st
No such fields
Uploading bl531_saxs_mono_angle_deg
No such fields
Uploading bl531_saxs_mono_height_mm
No such fields
Uploading bl531_saxs_hs_slit_A
No such fields
Uploading bl531_saxs_hs_slit_B
No such fields
Uploading bl531_saxs_hs_slit_C
No such fields
Uploading bl531:saxs:hs_slit:D
No such fields
Uploading bl531_saxs_hs_slit_E
No such fields
Uploading bl531_saxs_hs_slit_F
No such fields
Uploading bl531_saxs_hs_slit_G
No such fields
Uploading bl531_saxs_hs_slit_H
No such fields
Uploading bl531_saxs_hs
No such fields
Uploading bl531_saxs_sample_slit_E
No such fields
Uploading bl531_saxs_sample_slit_F
No such fields
Uploading bl531_saxs_sample_slit_G
No such fields
Uploading bl531_saxs_sample_slit_H
No such fields
Uploading bl531_saxs_hxp_x_mm
No such fields
Uploading bl531_saxs_hxp_y_mm
No such fields
Uploading bl531_saxs_hxp_z_mm
No such fields
Uploadin

In [ ]:
# # push json db 
# dbname = 'epicsHappi_mod'
# # jsonpath = './epicsDB.json'
# # with open(jsonpath, 'r') as d:
# #     pvlist = json.load(d)

# pushData(username, pwd, dbname, df)

In [26]:
t.drop(labels=['_id'], axis = 1)

,pvname,pvname_alias,motor,unit,hight,low
0,bl531:saxs:M1:pitch,bl531:m101:pitch,yes,deg,1,0
1,bl531:saxs:M1:bend,bl531:m101:bend,yes,deg,1,0
2,bl531:saxs:M1:slit,bl531:m101:slit,yes,mm,100,0
3,bl531:saxs:mono:angle,bl531:dcm:angle,yes,deg,50,0
4,bl531:saxs:mono:height,bl531:dcm:height,yes,mm,100,0
5,bl531:saxs:esp301:x,vewport:m11,yes,mm,5,0
6,bl531:saxs:esp301:y,vewport:m12,yes,mm,5,0
7,bl531:saxs:esp301:z,vewport:m13,yes,mm,5,0
8,bl531:saxs:camera:basler,,no,None,0,0
9,bl531:saxs:camera:manta,,no,None,0,0


In [6]:
df = pd.DataFrame(list(collection.find()))

In [7]:
df

,_id,active,args,beamline,creation,detailed_screen,device_class,documentation,embedded_screen,engineering_screen,...,macros,name,parent,prefix,stand,system,type,z,unit,port
0,6412404f23d463e59014d013,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_m101_pitch_mm,None,bl531_esp300:m101_pitch_mm,None,None,OphydItem,-1.0,mm,"ttyUSB300, axis1"
1,6412405023d463e59014d014,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_m101_bend_um,None,bl531_esp300:m101_bend_um,None,None,OphydItem,-1.0,um,"ttyUSB300, axis2"
2,6412405023d463e59014d015,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_m101_slit_st,None,bl531_esp300:m101_slit_mm,None,None,OphydItem,-1.0,mm,"ttyUSB300, axis3"
3,6412405023d463e59014d016,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_mono_angle_deg,None,IOC:m1,None,None,OphydItem,-1.0,deg,"192.168.10.25, axis1"
4,6412405023d463e59014d017,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_mono_height_mm,None,IOC:m2,None,None,OphydItem,-1.0,mm,"192.168.10.25, axis2"
5,6412405023d463e59014d018,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_A,None,DMC01:A,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit A"
6,6412405023d463e59014d019,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_B,None,DMC01:B,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit B"
7,6412405023d463e59014d01a,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_C,None,DMC01:C,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit C"
8,6412405023d463e59014d01b,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531:saxs:hs_slit:D,None,DMC01:D,None,None,motor,-1.0,mm,"192.168.10.21, slit D"
9,6412405023d463e59014d01c,True,[{{prefix}}],5.3.1,Tue Feb 28 08:27:04 2023,None,ophyd.EpicsMotor,None,None,None,...,None,bl531_saxs_hs_slit_E,None,DMC01:E,None,None,OphydItem,-1.0,mm,"192.168.10.21, slit E"
